In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import pickle
import re
import json
# from sklearn import cross_validation as cv
from collections import Counter
# from tensorflow.python.ops import math_ops
import matplotlib.pyplot as plt
import time
import datetime

In [2]:
article_title = [ 'id', 'feedname', 'url', 'orig_html', 'title', 'bodytext', 'relevant', 'created', 'last_modified', 'news_sourse_id', 'author']
article = pd.read_csv('./newsarticles_article.csv', header=None, names=article_title, dtype={'orig_html': str, 'author': str})

D:\ProgramData\Anaconda3\envs\FinalProject\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
article.head()

id feedname                                                url  \
0  122140        A  http://abclocal.go.com/wls/story?section=news/...   
1  122141        A  http://abclocal.go.com/wls/story?section=news/...   
2  122142        F  http://www.myfoxchicago.com/story/23377975/she...   
3  122143        C  http://www.beachwoodreporter.com/tv/24_hours_w...   
4  122144        C  http://www.beachwoodreporter.com/music/the_wee...   

  orig_html                                              title  \
0       NaN  Ventra cards available for all users Monday | ...   
1       NaN  Chicago Bears win season opener over Cincinnat...   
2       NaN  Sheriff: Inmate stabs sheriff's deputy, escape...   
3       NaN                   24 Hours With Al Jazeera America   
4       NaN                        The Weekend In Chicago Rock   

                                            bodytext relevant  \
0  ![][1]\n\nSeptember 8, 2013 (CHICAGO) (WLS) --...        f   
1  ![AP][1]\n\nSeptember 8, 2013 (CHICAGO) --  Fa...        f   
2    * ###  Sheriff: Inmate stabs sheriff's deput...        f   
3  Returning news from exile.\n\n**Midnight:** _R...        f   
4  You shoulda been there.\n\n**1.** Chance The R...        f   

                         created                  last_modified  \
0  2013-09-09 14:09:06.165191+00  2013-09-23 14:38:23.080001+00   
1  2013-09-09 14:09:07.943208+00  2013-09-23 14:38:07.687984+00   
2  2013-09-09 14:09:51.837171+00  2014-02-25 22:23:20.605934+00   
3  2013-09-09 14:11:53.733228+00  2013-09-23 14:38:23.088423+00   
4  2013-09-09 14:11:54.776239+00  2013-09-23 14:38:23.093802+00   

   news_sourse_id author  
0              98    NaN  
1              98    NaN  
2             109    NaN  
3              99    NaN  
4              99    NaN

In [4]:
article['relevant'] = article['relevant'] == 't'
article.rename(columns={'id': 'article_id'}, inplace=True)
article.head()

article_id feedname                                                url  \
0      122140        A  http://abclocal.go.com/wls/story?section=news/...   
1      122141        A  http://abclocal.go.com/wls/story?section=news/...   
2      122142        F  http://www.myfoxchicago.com/story/23377975/she...   
3      122143        C  http://www.beachwoodreporter.com/tv/24_hours_w...   
4      122144        C  http://www.beachwoodreporter.com/music/the_wee...   

  orig_html                                              title  \
0       NaN  Ventra cards available for all users Monday | ...   
1       NaN  Chicago Bears win season opener over Cincinnat...   
2       NaN  Sheriff: Inmate stabs sheriff's deputy, escape...   
3       NaN                   24 Hours With Al Jazeera America   
4       NaN                        The Weekend In Chicago Rock   

                                            bodytext  relevant  \
0  ![][1]\n\nSeptember 8, 2013 (CHICAGO) (WLS) --...     False   
1  ![AP][1]\n\nSeptember 8, 2013 (CHICAGO) --  Fa...     False   
2    * ###  Sheriff: Inmate stabs sheriff's deput...     False   
3  Returning news from exile.\n\n**Midnight:** _R...     False   
4  You shoulda been there.\n\n**1.** Chance The R...     False   

                         created                  last_modified  \
0  2013-09-09 14:09:06.165191+00  2013-09-23 14:38:23.080001+00   
1  2013-09-09 14:09:07.943208+00  2013-09-23 14:38:07.687984+00   
2  2013-09-09 14:09:51.837171+00  2014-02-25 22:23:20.605934+00   
3  2013-09-09 14:11:53.733228+00  2013-09-23 14:38:23.088423+00   
4  2013-09-09 14:11:54.776239+00  2013-09-23 14:38:23.093802+00   

   news_sourse_id author  
0              98    NaN  
1              98    NaN  
2             109    NaN  
3              99    NaN  
4              99    NaN

In [5]:
usercoding_title = ['id', 'date', 'relevant', 'article_id', 'user_id', 'locations']
usercoding = pd.read_csv('./newsarticles_usercoding.csv', header=None, names=usercoding_title)

In [6]:
usercoding.head()

id                           date relevant  article_id  user_id locations
0   1  2017-04-30 23:51:50.229747+00        f           1      NaN        []
1   2  2017-04-30 23:51:50.238775+00        f           2      NaN        []
2   3  2017-04-30 23:51:50.243216+00        f           3      NaN        []
3   4  2017-04-30 23:51:50.247388+00        f           4      NaN        []
4   5  2017-04-30 23:51:50.251488+00        f           5      NaN        []

In [7]:
usercoding.locations.unique()

array(['[]',
       '[{"start":211,"end":227,"text":"3252 W. 63rd St."},{"start":674,"end":688,"text":"600 W. 37th St"}]',
       '[{"start":189,"end":221,"text":"500 block of North State Street,"},{"start":73,"end":88,"text":"Near North Side"}]',
       ...,
       '[{"start":269,"end":288,"text":"U.S. Cellular Field"},{"start":2233,"end":2247,"text":"Glendale, Ariz"}]',
       '[{"start":71,"end":96,"text":"South and Near West sides"},{"start":181,"end":210,"text":"300 block of East 47th Street"},{"start":218,"end":242,"text":"Bronzeville neighborhood"},{"start":498,"end":512,"text":"Near West Side"},{"start":589,"end":622,"text":"1800 block of West Maypole Avenue"}]',
       '[{"start":83,"end":93,"text":"Englew ood"},{"start":114,"end":124,"text":"South Side"},{"start":188,"end":222,"text":"5500 block of South Emerald Avenue"}]'], dtype=object)

In [8]:
usercoding.locations = usercoding.locations.apply(lambda x: json.loads(x))

In [9]:
uc_categories_title = ['id', 'usercoding_id', 'category_id']
uc_categories = pd.read_csv( './newsarticles_usercoding_categories.csv', header=None, names=uc_categories_title )

In [10]:
uc_categories.head()

id  usercoding_id  category_id  article_id
0   1             15            2           1
1   2             15            5           2
2   3             15           13           3
3   4             25           17           4
4   5             25           26           5